In [1]:
import pandas as pd
import json
import sbg_driver
import yaml
import collections
from tqdm import tqdm
import os
from datetime import datetime
import numpy as np

In [2]:
# bag_file = "./train_dataset/outside/rosbag2_2023_08_29-10_08_36_0.db3"
# bag_file = "./train_dataset/slam_and_nav2/rosbag2_2023_08_29-13_03_38_0.db3"
bag_file = "./train_dataset/clear_data/rosbag2_2023_09_04-11_56_58_0.db3"

bag_dir = os.path.dirname(bag_file)

In [3]:
dir_points = os.path.join(bag_dir, "__points")
dir_geo = os.path.join(bag_dir, "__imu_sensor__sbg__gps_pos")

In [4]:
files_points = sorted([os.path.join(dir_points, fname) for fname in os.listdir(dir_points) if ".json" in fname],
                     key = lambda x: int(x.split("__points_rowid_")[1].split(".json")[0]))
files_geo = sorted([os.path.join(dir_geo, fname) for fname in os.listdir(dir_geo) if ".json" in fname], 
                   key=lambda x: int(x.split("__imu_sensor__sbg__gps_pos_rowid_")[1].split(".json")[0]))

In [5]:
df_geo = []
for file_geo in files_geo:    
    with open(file_geo) as user_file:
      file_contents = user_file.read()
    geo_data = json.loads(file_contents)
    df_row = {}
    df_row.update({"geo_file_path": file_geo})
    sec = geo_data["data"]["header"]["stamp"]["sec"]*1000000000
    nanosec = geo_data["data"]["header"]["stamp"]["nanosec"]
    df_row.update({"timestamp": sec+nanosec})
    df_row.update({"latitude": geo_data["data"]["latitude"]})
    df_row.update({"longitude": geo_data["data"]["longitude"]})
    df_row.update({"altitude": geo_data["data"]["altitude"]})
    df_geo.append(df_row.copy())
    # break

In [6]:
df_points = []
for file_points in files_points: 
    with open(file_points) as user_file:
      file_contents = user_file.read()
    points_data = json.loads(file_contents)
    df_row = {}
    df_row.update({"points_file_path": file_points})
    sec = points_data["data"]["header"]["stamp"]["sec"]*1000000000
    nanosec = points_data["data"]["header"]["stamp"]["nanosec"]
    df_row.update({"timestamp": sec+nanosec})
    df_points.append(df_row.copy())
    
    # break

In [7]:
df_geo = pd.DataFrame(df_geo)
df_points = pd.DataFrame(df_points)

In [8]:
compare_df = []
for row_points_id, row_points in df_points.iterrows():
    df_row = row_points.to_dict()
    timestamp_points = df_row["timestamp"]
    id_row_geo = np.abs(df_geo["timestamp"].to_numpy() - timestamp_points).argmin()
    geo_data = df_geo.iloc[id_row_geo]
    df_row.update({"geo_file_path": geo_data["geo_file_path"]})
    df_row.update({"latitude": geo_data["latitude"]})
    df_row.update({"longitude": geo_data["longitude"]})
    df_row.update({"altitude": geo_data["altitude"]})
    compare_df.append(df_row.copy())

In [9]:
compare_df = pd.DataFrame(compare_df)

In [12]:
compare_df

,points_file_path,timestamp,geo_file_path,latitude,longitude,altitude
0,./train_dataset/clear_data/__points/__points_r...,1693828618976335949,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681140,37.334718,183.917293
1,./train_dataset/clear_data/__points/__points_r...,1693828619076404671,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681140,37.334718,183.912093
2,./train_dataset/clear_data/__points/__points_r...,1693828619176689992,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681140,37.334718,183.912093
3,./train_dataset/clear_data/__points/__points_r...,1693828619276746839,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681140,37.334718,183.890893
4,./train_dataset/clear_data/__points/__points_r...,1693828619376756289,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681140,37.334718,183.890893
...,...,...,...,...,...,...
3502,./train_dataset/clear_data/__points/__points_r...,1693828983569936510,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681175,37.334719,185.949099
3503,./train_dataset/clear_data/__points/__points_r...,1693828983669842071,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681175,37.334719,185.939415
3504,./train_dataset/clear_data/__points/__points_r...,1693828983769841941,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681175,37.334719,185.939415
3505,./train_dataset/clear_data/__points/__points_r...,1693828983869998621,./train_dataset/clear_data/__imu_sensor__sbg__...,55.681175,37.334719,185.935499


In [11]:
compare_df.to_csv(os.path.join(bag_dir, "points_geo.csv"),index=False)